- To work, it requires videoseal to be installed and to be in the sylvestre/latent branch of it

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np

from videoseal.evals.full import setup_model_from_checkpoint

CompressAI package not found. Install with pip install compressai
Diffusers package not found. Install with pip install diffusers


/private/home/valeriu/.conda/envs/omnisealbench_all/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


EfficientViT not found.
EfficientViT not found. Make sure to install the efficientvit package.


In [3]:
# Loading the message.
msg = torch.ones((1, 64))

In [4]:
import glob
import os
import random
import omnisealbench
import tempfile

In [5]:
from omnisealbench.data.image import get_image_paths
from omnisealbench.utils.common import tensor_to_message, get_device

In [6]:
# Get all generated wm images in the parent directory.
parent_dir = "/private/home/sylvestre/avseal/avseal/logs/2021-04-03T19-39-50_cin_transformer2/samples/top_k_600_temp_1.00_top_p_0.92/0"

db = get_image_paths(parent_dir)
len(db)

16050

#### Prepare watermark .cache directory
 - as expected for the omnisealbench detection evaluation

In [8]:
import os
import json
import shutil as sh
from pathlib import Path
import uuid

cache_dir = Path(f"/tmp/latent_watermarking_{str(uuid.uuid4())}")
cache_dir.mkdir(exist_ok=True)
cache_dir

PosixPath('/tmp/latent_watermarking_1203c4bd-b82f-440a-a612-ddeef757dcfd')

In [13]:
num_samples = 100

tensor_to_message(msg[0].to(torch.int))

'1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1'

- for each of the element in the dataset we need the original data, the watermarked data and the watermark message

In [14]:
secret_message = msg[0].to(torch.int)

for i, sample in enumerate(db[:num_samples]):
    message_path = cache_dir/f"message_{i}.txt"
    watermark_path = cache_dir/f"watermark_{i}.png"
    data_path = cache_dir/f"data_{i}.png"

    with open(message_path, "w", encoding="utf-8") as f:
        f.write(tensor_to_message(secret_message))  # type: ignore
    
    sh.copy(sample, watermark_path)
    sh.copy(sample, data_path)

#### STEP 1: Define the detector only for our latent watermark model since we already have generated watermarked files
- The compatibility between model and the task:
- For the model to be evaluated in a detection task ("detection"), we must implement detect_watermark()

In [15]:
from typing import Dict, List
import torch
import torch.nn as nn
import torchvision.transforms as T

from omnisealbench.utils.detection import get_detection_and_decoded_keys



class LatentWatermark:
    
    model: nn.Module
    
    def __init__(self, model: torch.nn.Module, img_size: float = 256, nbits: int = 64, detection_bits: int = 16):
        self.model = model
            
        self.transform = T.Compose([
            T.Resize((img_size, img_size)),
        ])        
        
        # Each model should have an attribute 'nbits'. If the model does not have this attribute,
        # we must set the value `message_size` in the task. If Omniseal could not find information 
        # from either model or the task, it will raise the ValueError
        self.nbits = nbits
        self.detection_bits = detection_bits
    
    @torch.inference_mode()
    def detect_watermark(
        self,
        contents: torch.Tensor,
        detection_threshold: float = 0.0,
        message_threshold: float = 0.0,
    ) -> Dict[str, torch.Tensor]:
        # A detect_watermark() must have a specific signature:
        # Args:
        #  - contents: a torch.Tensor (with batch dimension at dim=0) or a list of torch.Tensor (each without batch dimension)
        #  - message_threshold: threshold used to convert the watermark output (probability
        #    of each bits being 0 or 1) into the binary n-bit message.
        #  - detection_threshold: threshold to convert the softmax output to binary indicating
        #    the probability of the content being watermarked
        # Returns:
        #  - a dictionary of with some keys such as:
        #    - "prediction": The prediction probability of the content being watermarked or not. The dimension should be 1 for batch size of `B`.
        #    - "message": The secret message of dimension `B x nbits`
        #    - "detection_bits": The list of bits reserved to calculating the detection accuracy.
        #   
        #    One of "prediction" and "detection_bits" must be provided. "message" is optional
        #    If "message" is returned, Omniseal Bench will compute message accuracy scores: "bit_acc", "word_acc", "p_value", "capacity", and "log10_p_value"
        #    Otherwise, these metrics will be skipped

        image_tensors = []
        for img in contents:
            img_tensor = self.transform(img).unsqueeze(0).to(get_device(self))
            image_tensors.append(img_tensor)
            
        image_tensors = torch.cat(image_tensors, dim=0)
        extracted_bits = self.model.detector(image_tensors)[:, 1:] > 0

        return get_detection_and_decoded_keys(
            extracted_bits,
            detection_bits=self.detection_bits,
            detection_threshold=detection_threshold,
            message_threshold=message_threshold,
        )

#### STEP 2: Define the builder function.
 - The function can have any parameters, but should contain at least one parameter "device" which defines which device the model object will be placed too.
 - It is advisable to have the parameters() of the model class __init__() match the arguments of this function.

In [16]:
def build_latent_watermark_model(ckpt_path: str, nbits: int = 64, detection_bits: int = 16, device: str = "cpu") -> LatentWatermark:    
    watermarker = setup_model_from_checkpoint(ckpt_path)    
    watermarker = watermarker.eval()
    watermarker = watermarker.to(device)
    
    return LatentWatermark(model=watermarker, img_size=256, nbits=nbits, detection_bits=detection_bits)

### Run the detection only task

In [17]:
from omnisealbench import task, get_model

In [20]:
detection_task = task(
    "detection",
    modality="image",
    seed=42,
    dataset_dir=str(cache_dir),
    original_image_pattern="data*.png",
    watermarked_image_pattern="watermark*.png",  # We just fake the watermarked images
    message_pattern="message_*.txt",
    # metrics="all",  # We can add one more quality metric that was not previously computed
    metrics=['lpips', 'psnr', 'ssim'],
    # result_dir="/tmp/detection_image",
    # overwrite=True,
    attacks=["comb", "gaussian_blur"],
    batch_size=2,
)

detector = get_model(
    build_latent_watermark_model, 
    as_type="detector", device="cuda", 
    ckpt_path='/checkpoint/valeriu/weights/latent_watermarker/checkpoint600.pth',
    nbits=64,
    detection_bits=16,
)

avg_metrics, raw_results = detection_task(detector)

Model loaded successfully from /checkpoint/valeriu/weights/latent_watermarker/checkpoint600.pth with message: _IncompatibleKeys(missing_keys=[], unexpected_keys=['autoencoder.model.encoder.conv_in.weight', 'autoencoder.model.encoder.conv_in.bias', 'autoencoder.model.encoder.down.0.block.0.norm1.weight', 'autoencoder.model.encoder.down.0.block.0.norm1.bias', 'autoencoder.model.encoder.down.0.block.0.conv1.weight', 'autoencoder.model.encoder.down.0.block.0.conv1.bias', 'autoencoder.model.encoder.down.0.block.0.norm2.weight', 'autoencoder.model.encoder.down.0.block.0.norm2.bias', 'autoencoder.model.encoder.down.0.block.0.conv2.weight', 'autoencoder.model.encoder.down.0.block.0.conv2.bias', 'autoencoder.model.encoder.down.0.block.1.norm1.weight', 'autoencoder.model.encoder.down.0.block.1.norm1.bias', 'autoencoder.model.encoder.down.0.block.1.conv1.weight', 'autoencoder.model.encoder.down.0.block.1.conv1.bias', 'autoencoder.model.encoder.down.0.block.1.norm2.weight', 'autoencoder.model.enco

/private/home/valeriu/.conda/envs/omnisealbench_all/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/private/home/valeriu/.conda/envs/omnisealbench_all/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Running ImageWatermarkAttacksAndDetection with attack: gaussian_blur__kernel_size_3
Running ImageWatermarkAttacksAndDetection with attack: gaussian_blur__kernel_size_5
Running ImageWatermarkAttacksAndDetection with attack: gaussian_blur__kernel_size_9
Running ImageWatermarkAttacksAndDetection with attack: gaussian_blur__kernel_size_13
Running ImageWatermarkAttacksAndDetection with attack: gaussian_blur__kernel_size_17
Running ImageWatermarkAttacksAndDetection with attack: comb


Note: here the averages should take in account/weight the number of variants

In [21]:
avg_metrics

{'watermark_det_score': AverageMetric(avg=0.9300892857142857, count=700, square=0.8720256696428571, avg_ci_fn=None),
 'watermark_det': AverageMetric(avg=0.9985714285714286, count=700, square=0.9985714285714286, avg_ci_fn=None),
 'fake_det_score': AverageMetric(avg=0.9300892857142857, count=700, square=0.8720256696428571, avg_ci_fn=None),
 'fake_det': AverageMetric(avg=0.9985714285714286, count=700, square=0.9985714285714286, avg_ci_fn=None),
 'bit_acc': AverageMetric(avg=0.9153869238921574, count=700, square=0.8445976291478626, avg_ci_fn=None),
 'word_acc': AverageMetric(avg=0.08571428571428572, count=700, square=0.08571428571428572, avg_ci_fn=None),
 'p_value': AverageMetric(avg=0.0047896499597762906, count=700, square=0.002258927683709689, avg_ci_fn=None),
 'capacity': AverageMetric(avg=30.414895470482964, count=700, square=1038.6499686100688, avg_ci_fn=None),
 'log10_p_value': AverageMetric(avg=-9.704717900215343, count=700, square=103.64760890944434, avg_ci_fn=None),
 'ssim': Avera

In [22]:
scores_df = detection_task.print_scores(raw_results)
scores_df

,watermark_det_score,watermark_det,fake_det_score,fake_det,bit_acc,word_acc,p_value,capacity,log10_p_value,ssim,lpips,psnr,decoder_time,qual_time,det_time,attack_time,idx,attack,attack_variant,cat
0,0.9375,True,0.9375,True,1.000000,True,3.552714e-15,48.000000,-14.449440,1.0,0.0,inf,0.0053,0.8697,0.0019,0.0000,0,identity,default,none
1,1.0000,True,1.0000,True,1.000000,True,3.552714e-15,48.000000,-14.449440,1.0,0.0,inf,0.0053,0.8697,0.0019,0.0000,1,identity,default,none
2,0.8750,True,0.8750,True,0.937500,False,6.562928e-11,31.810078,-10.182902,1.0,0.0,inf,0.0039,0.2102,0.0007,0.0000,2,identity,default,none
3,1.0000,True,1.0000,True,0.937500,False,6.562928e-11,31.810078,-10.182902,1.0,0.0,inf,0.0039,0.2102,0.0007,0.0000,3,identity,default,none
4,1.0000,True,1.0000,True,0.854167,False,3.120204e-07,19.232918,-6.505817,1.0,0.0,inf,0.0041,0.2067,0.0008,0.0000,4,identity,default,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,0.8125,True,0.8125,True,0.854167,False,3.120204e-07,19.232918,-6.505817,NaN,NaN,NaN,0.0034,0.0000,0.0005,0.0019,95,comb,default,Mixed
696,0.6875,True,0.6875,True,0.770833,False,1.111225e-04,10.725326,-3.954198,NaN,NaN,NaN,0.0034,0.0000,0.0005,0.0019,96,comb,default,Mixed
697,0.8125,True,0.8125,True,0.916667,False,7.569163e-10,28.136793,-9.120952,NaN,NaN,NaN,0.0034,0.0000,0.0005,0.0019,97,comb,default,Mixed
698,1.0000,True,1.0000,True,0.687500,False,6.641641e-03,4.990165,-2.177725,NaN,NaN,NaN,0.0034,0.0000,0.0005,0.0019,98,comb,default,Mixed


In [23]:
from omnisealbench.utils.analysis import aggregate_by_attacks, aggregate_by_attack_variants

In [24]:
aggregate_by_attacks(scores_df)

,attack,cat,psnr,ssim,lpips,bit_acc,log10_p_value,TPR,FPR,watermark_det_score,fake_det_score,capacity
0,comb,Mixed,NaN,NaN,NaN,0.756875,-4.069825,0.99,0.99,0.808750,0.808750,11.274060
1,gaussian_blur,Visual,inf,1.0,0.0,0.941625,-10.627832,1.00,1.00,0.949625,0.949625,33.546480
2,identity,none,inf,1.0,0.0,0.942708,-10.724038,1.00,1.00,0.953750,0.953750,33.897807


In [25]:
aggregate_by_attack_variants(scores_df)

,attack,attack_variant,cat,psnr,ssim,lpips,bit_acc,log10_p_value,TPR,FPR,watermark_det_score,fake_det_score,capacity
0,comb,default,Mixed,NaN,NaN,NaN,0.756875,-4.069825,0.99,0.99,0.808750,0.808750,11.274060
1,gaussian_blur,kernel_size_13,Visual,inf,1.0,0.0,0.940833,-10.572671,1.00,1.00,0.946250,0.946250,33.348233
2,gaussian_blur,kernel_size_17,Visual,inf,1.0,0.0,0.937500,-10.398696,1.00,1.00,0.945625,0.945625,32.723669
3,gaussian_blur,kernel_size_3,Visual,inf,1.0,0.0,0.943333,-10.750516,1.00,1.00,0.953750,0.953750,34.014827
4,gaussian_blur,kernel_size_5,Visual,inf,1.0,0.0,0.942500,-10.682998,1.00,1.00,0.952500,0.952500,33.742841
5,gaussian_blur,kernel_size_9,Visual,inf,1.0,0.0,0.943958,-10.734281,1.00,1.00,0.950000,0.950000,33.902831
6,identity,default,none,inf,1.0,0.0,0.942708,-10.724038,1.00,1.00,0.953750,0.953750,33.897807
